Every movies plot have been analysed by the Stanford CoreNLP, we get search through it using the element tree from xml. We just have to give the movie id. 

In [41]:
import xml.etree.ElementTree as ET

DATA = '../corenlp_plot_summaries/'

def parse_nlp(id : str) -> ET :
    tree = ET.parse(DATA + id + '.xml')
    return tree.getroot()

We want to be able to translate the freebase id to their meaning to make onsightful analysis on our data.

In [42]:
import requests
import urllib

def translate_freebase(freebase_id : str) -> str:
    query_content = """
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    SELECT  ?sLabel WHERE {
        ?s wdt:P646 "%s"
        SERVICE wikibase:label {
            bd:serviceParam wikibase:language "en" .
        }
    }
    """ % freebase_id
    
    query = "https://query.wikidata.org/bigdata/namespace/wdq/sparql?format=json&query=" + urllib.parse.quote_plus(query_content)
    response = requests.get(query)
    return response.json()['results']['bindings'][0]['sLabel']['value']
